In [20]:
from deltalake import DeltaTable

dt = DeltaTable("s3://delta/bronze/iot_poc_v2", None, {
    "aws_default_region": "us-east-2",
    "aws_secret_access_key": "minioadmin",
    "allow_http": "true",
    "aws_access_key_id": "minioadmin",
    "aws_endpoint": "http://localhost:9000"
})



In [27]:
dir(dt)

['_DeltaTable__stringify_partition_values',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_metadata',
 '_storage_options',
 '_table',
 'create_checkpoint',
 'file_uris',
 'files',
 'files_by_partitions',
 'from_data_catalog',
 'get_add_actions',
 'history',
 'load_version',
 'load_with_datetime',
 'metadata',
 'optimize',
 'protocol',
 'pyarrow_schema',
 'schema',
 'table_uri',
 'to_pandas',
 'to_pyarrow_dataset',
 'to_pyarrow_table',
 'update_incremental',
 'vacuum',
 'version']

In [29]:
dt.to_pandas().values

array([[b' \xff?\x8f\x1c\x191\x9b\xf6\xd26n\xe2,\xd3\xa0\xdd\x92\xcb\xd9\x7f\x83:\x19\xda\xd3\x97\x1a\xa4\xb2\x9a\xf4',
        {'received_timestamp': Decimal('1674765710996'), 'location': '631243964719518719', 'hex_scale': 1637, 'report': {'pub_key': b'\x00\x8bT\xb7l(\xa22\x1a\x98\xb3\xf1:~\x9a\xaa\xc4\xff\xe5\x00\x9d\x8eQ\x01\x87\x87\x9a\x8b\xabJ\xbfl]', 'local_entropy': b'\xfb0z\x00', 'remote_entropy': b'\xb9T\x12\xa7(C{_ls\x17\x81\xd4\n\x10\xeb\x1a\x9a\xa9\xf73\xb5y\x8d"\x93\xbb\xc6J\xc2\xc8\x94', 'data': b'+\xc3ag\xaf\xae\x87\xd7\xa1\x03', 'frequency': Decimal('903900000'), 'channel': 0, 'datarate': 'helium.data_rate.SF7BW125', 'tx_power': 27, 'timestamp': Decimal('1674765710632451368'), 'signature': b"0E\x02!\x00\xd5\xc8\xd0\x1d\xcc?\xbb\x00}+\xda\x9a\xc8\xa1\xe8J\xbc\x82u\xa8\xb9\xbb\xadU\x1c\\\xb8\xd9\xe1\xc8!\xe4\x02 g\xa4av}\xbfS\xc4\xfc\x95j,\xbb\x90\x8e\x1c\xfb'\xe0\x1b2\x19\x9bzoB\xcf\ri\xe0\xba\x8a", 'tmst': 0}, 'reward_unit': 0, 'gain': 0, 'elevation': 0},
        array(

In [1]:
import pyspark
from delta.pip_utils import configure_spark_with_delta_pip

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

23/06/13 10:18:30 WARN Utils: Your hostname, Noahs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.4.32 instead (on interface en0)
23/06/13 10:18:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/nprince/.ivy2/cache
The jars for the packages stored in: /Users/nprince/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d6075321-f879-49fc-ac23-8c37317ae4b3;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 145ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0

In [9]:
table = spark.sql("CREATE TABLE IF NOT EXISTS delta.`/tmp/delta-table` USING DELTA AS SELECT col1 as id FROM VALUES 0,1,2,3,4;")

In [15]:
spark.conf.set("spark.databricks.delta.schema.autoMerge", "True")

In [7]:
extra_col = spark.sql("SELECT 5 as id, 2 as new_col")

In [18]:
extra_col.write.format("delta").mode("append").option("mergeSchema", "true").save("/tmp/delta-table")